In [28]:
import pandas as pd
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
from sklearn.metrics import pairwise_distances
import pickle
from pprint import pprint
import json

In [49]:
def similar_dishes(recipeId, df, model_name ):
    model = pickle.load(open(model_name, 'rb'))
    df2 = model.transform(df['RecipeIngredientParts'])
    # Make Sparse Matrix
    sparse_df = sparse.csr_matrix(df2)   
    # Calculate distances bewteen vectors from sparse matrix (cosine)
    distances = pairwise_distances(sparse_df, metric='cosine')
    
    recommender_df = pd.DataFrame(distances, 
                                  columns=df['RecipeId'], 
                                  index=df['RecipeId'])

    df3=pd.concat([df[['Name', 'RecipeId']].reset_index(),recommender_df.reset_index()],axis=1)

    return df3[[recipeId, 'Name', 'RecipeId']].sort_values(by=recipeId).head(6)[1:].iloc[:, 3].values.tolist()

def similar_dishes_json(df, similar_dishes):
    return json.loads(df.loc[df['RecipeId'].isin(similar_dishes)][['RecipeId','Name','TotalTime', 'Images', 'RecipeIngredientParts', 'FatContent', 'Calories','SaturatedFatContent','CholesterolContent','SodiumContent' ,'CarbohydrateContent','FiberContent','SugarContent','ProteinContent','RecipeServings','RecipeInstructions']].to_json(orient="records"))

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [53]:
df = pd.read_csv('short_recipes.csv')
sim_dishes = similar_dishes(1145, df, 'transformer.pkl')
pprint(similar_dishes_json(df, sim_dishes))

[{'Calories': 373.4,
  'CarbohydrateContent': 13.5,
  'CholesterolContent': 103.5,
  'FatContent': 21.8,
  'FiberContent': 2.3,
  'Images': '[0]',
  'Name': 'Chinese Chicken',
  'ProteinContent': 28.0,
  'RecipeId': 193,
  'RecipeIngredientParts': '[chicken, tomato sauce, soy sauce, dry sherry, '
                           'white vinegar, chili powder, garlic, green ginger]',
  'RecipeInstructions': '[Wash and dry chicken., Combine the rest of the '
                        'ingredients together and mix well., Add the chicken, '
                        'coat well with the marinade.  Allow to stand several '
                        'hours or overnight in refrigerator., Place chicken in '
                        'a baking dish and pour over marinade.  Cover dish '
                        'with foil., Bake in a moderate oven 1 1/2 hours, '
                        'remove foil, bake a further 30 minutes, brushing '
                        'frequently with marinade.]',
  'RecipeServings': 6.

In [57]:
l = []


[]

In [51]:
pprint(json.loads(df.loc[df['RecipeId'].isin(sim_dishes)][[
    'RecipeId',
    'Name',
    'TotalTime', 
    'Images', 
    'RecipeIngredientParts', 
    'FatContent', 
    'Calories',
    'SaturatedFatContent',
    'CholesterolContent',
    'SodiumContent' ,
    'CarbohydrateContent',
    'FiberContent',
    'SugarContent',
    'ProteinContent',
    'RecipeServings',
    'RecipeInstructions']].to_json(orient="records")))

[{'Calories': 373.4,
  'CarbohydrateContent': 13.5,
  'CholesterolContent': 103.5,
  'FatContent': 21.8,
  'FiberContent': 2.3,
  'Images': '[0]',
  'Name': 'Chinese Chicken',
  'ProteinContent': 28.0,
  'RecipeId': 193,
  'RecipeIngredientParts': '[chicken, tomato sauce, soy sauce, dry sherry, '
                           'white vinegar, chili powder, garlic, green ginger]',
  'RecipeInstructions': '[Wash and dry chicken., Combine the rest of the '
                        'ingredients together and mix well., Add the chicken, '
                        'coat well with the marinade.  Allow to stand several '
                        'hours or overnight in refrigerator., Place chicken in '
                        'a baking dish and pour over marinade.  Cover dish '
                        'with foil., Bake in a moderate oven 1 1/2 hours, '
                        'remove foil, bake a further 30 minutes, brushing '
                        'frequently with marinade.]',
  'RecipeServings': 6.

In [39]:
df.loc[df['RecipeId'].isin(sim_dishes)][[
    'RecipeId',
    'TotalTime', 
    'Images', 
    'RecipeIngredientParts', 
    'RecipeIngredientParts', 
    'FatContent', 
    'Calories',
    'SaturatedFatContent',
    'CholesterolContent',
    'SodiumContent' ,
    'CarbohydrateContent',
    'FiberContent',
    'SugarContent',
    'ProteinContent',
    'RecipeServings',
    'RecipeInstructions']]

,RecipeId,TotalTime,Images,RecipeIngredientParts,RecipeIngredientParts,FatContent,Calories,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions
144,193,PT1H55M,[0],"[chicken, tomato sauce, soy sauce, dry sherry,...","[chicken, tomato sauce, soy sauce, dry sherry,...",21.8,373.4,6.3,103.5,491.8,13.5,2.3,1.2,28.0,6.0,"[Wash and dry chicken., Combine the rest of th..."
440,523,PT15M,[0],"[sugar, salt, dry white wine, honey, Worcester...","[sugar, salt, dry white wine, honey, Worcester...",0.2,678.0,0.0,0.0,1226.0,171.7,1.4,147.2,1.2,NaN,"[Mix together sugar, apricot jam, salt, aprico..."
533,620,PT30M,[0],"[chicken, carrot, cabbage, celery, bean sprout...","[chicken, carrot, cabbage, celery, bean sprout...",18.3,504.5,5.1,90.6,1821.1,53.9,4.6,20.9,30.4,8.0,"[Boil chicken., Debone set aside. Cut up celer..."
803,1376,PT5M,[0],white wine vinegar,white wine vinegar,0.6,88.7,0.0,0.0,0.0,21.7,3.4,19.1,2.1,NaN,[In a glass or stainless steel sauce pan heat ...
804,1379,PT0S,[https://img.sndimg.com/food/image/upload/w_55...,"[garlic, fresh basil, white wine vinegar]","[garlic, fresh basil, white wine vinegar]",0.3,29.6,0.0,0.0,3.8,5.8,1.9,0.2,1.9,NaN,[In a glass or stainless steel sauce pan heat ...


In [ ]:
Calories                    1001 non-null   float64
 14  FatContent                  1001 non-null   float64
 15  SaturatedFatContent         1001 non-null   float64
 16  CholesterolContent          1001 non-null   float64
 17  SodiumContent               1001 non-null   float64
 18  CarbohydrateContent         1001 non-null   float64
 19  FiberContent                1001 non-null   float64
 20  SugarContent                1001 non-null   float64
 21  ProteinContent              1001 non-null   float64
 22  RecipeServings              689 non-null    float64
 23  RecipeInstructions

In [139]:
df1

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522512,541379,Meg's Fresh Ginger Gingerbread,2002090414,rdsxc,PT35M,PT1H,PT1H35M,2020-12-22T15:27:00Z,Make and share this Meg's Fresh Ginger Gingerb...,character(0),...,7.6,54.4,278.2,48.5,0.8,22.8,3.9,8.0,1 8x8 cake pan,"c(""Preheat oven to 350&deg;F Grease an 8x8 cak..."
522513,541380,Roast Prime Rib au Poivre with Mixed Peppercorns,211566,Denver cooks,PT3H,PT30M,PT3H30M,2020-12-22T15:32:00Z,"White, black, green, and pink peppercorns add ...","""https://img.sndimg.com/food/image/upload/w_55...",...,71.4,433.8,766.3,3.2,0.7,0.1,117.0,8.0,1 Roast,"c(""Position rack in center of oven and preheat..."
522514,541381,Kirshwasser Ice Cream,2001131545,Jonathan F.,PT3H,PT1H,PT4H,2020-12-22T15:33:00Z,Make and share this Kirshwasser Ice Cream reci...,character(0),...,72.6,470.9,192.5,33.9,0.0,17.3,12.8,6.0,NaN,"c(""heat half and half and heavy cream to a sim..."
522515,541382,Quick & Easy Asian Cucumber Salmon Rolls,2001004241,CLUBFOODY,NaN,PT15M,PT15M,2020-12-22T22:11:00Z,"Extremely quick and easy to make, these are gr...","""https://img.sndimg.com/food/image/upload/w_55...",...,0.1,2.9,100.5,0.3,0.0,0.1,2.4,NaN,20 rolls,"c(""In a small bowl, combine mayo and wasabi pa..."


In [140]:
df1 = df1.drop(columns=['AuthorId', 'AuthorName', 'DatePublished', 'RecipeYield'], axis=1)

In [141]:
df1

,RecipeId,Name,CookTime,PrepTime,TotalTime,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,...,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,PT24H,PT45M,PT24H45M,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",Frozen Desserts,"c(""Dessert"", ""Low Protein"", ""Low Cholesterol"",...","c(""4"", ""1/4"", ""1"", ""1"")",...,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,PT25M,PT4H,PT4H25M,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",Chicken Breast,"c(""Chicken Thigh & Leg"", ""Chicken"", ""Poultry"",...","c(""1"", ""4"", ""2"", ""2"", ""8"", ""1/4"", ""8"", ""1/2"", ...",...,58.8,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,PT5M,PT30M,PT35M,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",Beverages,"c(""Low Protein"", ""Low Cholesterol"", ""Healthy"",...","c(""1 1/2"", ""1"", NA, ""1 1/2"", NA, ""3/4"")",...,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,PT20M,PT24H,PT24H20M,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",Soy/Tofu,"c(""Beans"", ""Vegetable"", ""Low Cholesterol"", ""We...","c(""12"", ""1"", ""2"", ""1"", ""10"", ""1"", ""3"", ""2"", ""2...",...,24.0,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,PT30M,PT20M,PT50M,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",Vegetable,"c(""Low Protein"", ""Vegan"", ""Low Cholesterol"", ""...","c(""46"", ""4"", ""1"", ""2"", ""1"")",...,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,"c(""Mix everything together and bring to a boil..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522512,541379,Meg's Fresh Ginger Gingerbread,PT35M,PT1H,PT1H35M,Make and share this Meg's Fresh Ginger Gingerb...,character(0),Dessert,"""< 4 Hours""","c(""3"", ""1/2"", ""1/2"", ""1/4"", ""1/4"", ""1"", ""1/4"",...",...,12.5,7.6,54.4,278.2,48.5,0.8,22.8,3.9,8.0,"c(""Preheat oven to 350&deg;F Grease an 8x8 cak..."
522513,541380,Roast Prime Rib au Poivre with Mixed Peppercorns,PT3H,PT30M,PT3H30M,"White, black, green, and pink peppercorns add ...","""https://img.sndimg.com/food/image/upload/w_55...",Very Low Carbs,"c(""High Protein"", ""High In..."", ""< 4 Hours"")","c(""9"", ""2"", ""4"", ""2"", ""1/3"", ""3 1/2"", ""1/3"")",...,172.4,71.4,433.8,766.3,3.2,0.7,0.1,117.0,8.0,"c(""Position rack in center of oven and preheat..."
522514,541381,Kirshwasser Ice Cream,PT3H,PT1H,PT4H,Make and share this Kirshwasser Ice Cream reci...,character(0),Ice Cream,"c(""Dessert"", ""< 4 Hours"")","c(""3"", ""3"", ""1/2"", ""1/2"", ""2"", ""2"")",...,117.2,72.6,470.9,192.5,33.9,0.0,17.3,12.8,6.0,"c(""heat half and half and heavy cream to a sim..."
522515,541382,Quick & Easy Asian Cucumber Salmon Rolls,NaN,PT15M,PT15M,"Extremely quick and easy to make, these are gr...","""https://img.sndimg.com/food/image/upload/w_55...",Canadian,"c(""< 15 Mins"", ""Easy"")","c(""4"", ""1/4"", ""1"", ""1/2"", ""8 1/2"", ""40"")",...,0.6,0.1,2.9,100.5,0.3,0.0,0.1,2.4,NaN,"c(""In a small bowl, combine mayo and wasabi pa..."


In [147]:
df = df1[:1001]

In [158]:
df.to_csv('short_recipes.csv')

In [148]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   RecipeId                    1001 non-null   int64  
 1   Name                        1001 non-null   object 
 2   CookTime                    626 non-null    object 
 3   PrepTime                    1001 non-null   object 
 4   TotalTime                   1001 non-null   object 
 5   Description                 1001 non-null   object 
 6   Images                      1001 non-null   object 
 7   RecipeCategory              1001 non-null   object 
 8   Keywords                    991 non-null    object 
 9   RecipeIngredientQuantities  1000 non-null   object 
 10  RecipeIngredientParts       1001 non-null   object 
 11  AggregatedRating            806 non-null    float64
 12  ReviewCount                 816 non-null    float64
 13  Calories                    1001 

In [149]:
df["RecipeIngredientParts"] = df["RecipeIngredientParts"].apply(lambda x: x.replace("c(", "[").replace('"', "").replace(")", "]").replace("character(", "["))

C:\Users\Salut Tudor!\AppData\Local\Temp\ipykernel_15588\913486564.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["RecipeIngredientParts"] = df["RecipeIngredientParts"].apply(lambda x: x.replace("c(", "[").replace('"', "").replace(")", "]").replace("character(", "["))


In [177]:
for col in df.columns[5:]:
    if df[col].dtype == 'O':
        print(col)
        df[col] = df[col].apply(lambda x: str(x).replace("c(", "[").replace('"', "").replace(")", "]").replace("character(", "["))

Description
Images
RecipeCategory
Keywords
RecipeIngredientQuantities
RecipeIngredientParts
RecipeInstructions
unique


C:\Users\Salut Tudor!\AppData\Local\Temp\ipykernel_15588\1902450361.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: str(x).replace("c(", "[").replace('"', "").replace(")", "]").replace("character(", "["))


In [179]:
df.to_csv('short_recipes.csv')

In [172]:
df['Name'].dtype == 'O'

True

In [150]:
# Make lists into individual strings
df['RecipeIngredientParts'] = df['RecipeIngredientParts'].map(''.join)
# Vectorize - New DataFrame for transformed data
tvec = TfidfVectorizer()
tvec.fit(df['RecipeIngredientParts'])
df2 = tvec.transform(df['RecipeIngredientParts'])
# Make Sparse Matrix
sparse_df = sparse.csr_matrix(df2)

C:\Users\Salut Tudor!\AppData\Local\Temp\ipykernel_15588\81974805.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RecipeIngredientParts'] = df['RecipeIngredientParts'].map(''.join)


In [155]:

pickle.dump(tvec, open('transformer.pkl', 'wb'))

In [154]:
# Calculate distances bewteen vectors from sparse matrix (cosine)
distances = pairwise_distances(sparse_df, metric='cosine')
# Engineer unique label data from original DataFrame
df['unique'] = df['Name'] + df['RecipeId'].astype(str)
# Make labeled DataFrame as the basis for our recommender
recommender_df = pd.DataFrame(distances, 
                              columns=df['unique'], 
                              index=df['unique'])

df3=pd.concat([df['Name'].reset_index(),recommender_df.reset_index()],axis=1)

df3[['Biryani39','Name']].sort_values(by='Biryani39').head(6)[1:]

C:\Users\Salut Tudor!\AppData\Local\Temp\ipykernel_15588\3259033484.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['unique'] = df['Name'] + df['RecipeId'].astype(str)


,Biryani39,Name
380,0.577354,Lentil Dal
112,0.637650,Chicken Curry
801,0.685186,Ground Beef Stuffed Green Bell Peppers With Ch...
671,0.687711,Barbecue Chicken
689,0.689912,Onion Pie


In [ ]:
def similar_dishes(name, df):
    # Make lists into individual strings
    df = df.map(''.join)
    # Vectorize - New DataFrame for transformed data
    tvec = TfidfVectorizer()
    tvec.fit(df)
    df2 = tvec.transform(df)
    # Make Sparse Matrix
    sparse_df = sparse.csr_matrix(df2)
    
    # Calculate distances bewteen vectors from sparse matrix (cosine)
    distances = pairwise_distances(sparse_df, metric='cosine')
    # Engineer unique label data from original DataFrame
    df['unique'] = df['Name'] + df['RecipeId'].astype(str)
    # Make labeled DataFrame as the basis for our recommender
    recommender_df = pd.DataFrame(distances, 
                                  columns=df['unique'], 
                                  index=df['unique'])

    df3=pd.concat([df['Name'].reset_index(),recommender_df.reset_index()],axis=1)
    
    

In [95]:
df['RecipeIngredientParts'].map(' '.join)

0       [ b l u e b e r r i e s ,   g r a n u l a t e ...
1       [ s a f f r o n ,   m i l k ,   h o t   g r e ...
2       [ s u g a r ,   l e m o n s ,   r i n d   o f ...
3       [ e x t r a   f i r m   t o f u ,   e g g p l ...
4       [ p l a i n   t o m a t o   j u i c e ,   c a ...
                              ...                        
996     [ m a r g a r i n e ,   V e l v e e t a   c h ...
997     [ f l o u r ,   b a k i n g   s o d a ,   s a ...
998     [ f l o u r ,   c o r n m e a l ,   b a k i n ...
999     [ y e l l o w   c o r n m e a l ,   w h i t e ...
1000    [ c o r n m e a l ,   g r i t s ,   w a t e r ...
Name: RecipeIngredientParts, Length: 1001, dtype: object

In [73]:
df['RecipeIngredientParts'].apply(literal_eval)

0       [blueberries, granulated sugar, vanilla yogurt...
1       [saffron, milk, hot green chili peppers, onion...
2       [sugar, lemons, rind of, lemon, zest of, fresh...
3       [extra firm tofu, eggplant, zucchini, mushroom...
4       [plain tomato juice, cabbage, onion, carrots, ...
                              ...                        
996     [margarine, Velveeta cheese, vanilla, powdered...
997     [flour, baking soda, salt, light brown sugar, ...
998     [flour, cornmeal, baking powder, salt, garlic ...
999     [yellow cornmeal, white cornmeal, sugar, bakin...
1000    [cornmeal, grits, water, butter, heavy cream, ...
Name: RecipeIngredientParts, Length: 1001, dtype: object

In [38]:
df["RecipeIngredientParts"][0].replace('c(', '[')


'["blueberries", "granulated sugar", "vanilla yogurt", "lemon juice")'

In [42]:
df["RecipeIngredientParts"] = df["RecipeIngredientParts"].apply(lambda x: x.replace("c(", "[").replace(")", "]"))

In [44]:
df["RecipeIngredientParts"][5]

'["graham cracker crumbs", "sugar", "butter", "sugar", "cornstarch", "salt", "milk", "vanilla extract", "water", "gelatin", "rum", "cream of tartar", "sugar"]'